# Data Preparation

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import psycopg2
import pandas as pd
import numpy as np
#import dbpass
pd.set_option('max_colwidth',0)

In [9]:
DBNAME = "opportunity_youth"
conn = psycopg2.connect(dbname=DBNAME, user="postgres")
#conn = psycopg2.connect(dbname=DBNAME, user="postgres", password=dbpass.postgrepass())

In [21]:
query = """
SELECT cow, esr, schl education_attained, fschp enrollment_status, agep age
FROM pums_2017
where puma between '11610' and '11615'
or puma between '11604' and '11605'
"""
df = pd.read_sql(query, conn)
df['employed'] = 1

df.loc[(df.cow == '3') | (df.esr == '9'),'employed'] = 0
df.loc[(df.cow == np.nan) & (df.esr == np.nan), 'employed'] = 0
df.drop(columns = ['cow','esr'], inplace = True)
df['education_attained'].fillna(df['education_attained'].mode()[0],inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45511 entries, 0 to 45510
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   education_attained  45511 non-null  object 
 1   enrollment_status   45511 non-null  object 
 2   age                 45511 non-null  float64
 3   employed            45511 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 1.4+ MB


In [22]:
df.head()

,education_attained,enrollment_status,age,employed
0,19,0,26.0,1
1,19,0,22.0,1
2,09,0,52.0,1
3,01,0,47.0,1
4,15,0,18.0,1


puma = region (11613 - 11615 for S. King County)

agep = age (16-24)

cow = employment (9 = unemployed)

fschp = school enrollment (1 = yes, 0 = no)

esr = employment recode (3 = unemployed)

fesrp employment recode (0 - not employed)

In [ ]:
conn.close()